# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트
[books] 트리의 앙상블

Page 263 

- 앙상블 모델 - 배깅 
- 랜덤 포레스트 모델을 여러개 사용하여 각 모델의 예측 값을 보팅을 통해 예측 
- 랜덤 포레스트(Random Forest)
- 부트스트랩 샘플(Bootstrap Sample) 
 * 샘플을 선택할 때 중복해서 선택가능 
 * 훈련 세트의 크기와 동일한 크기의 새믈을 선택 
 
- Scikit Learn 랜덤 포레스트는 기본적으로 100개의 결정트리를 훈련.
- RandomForestClassifier: 분류일때 각 트리의 클래스별 확율을 가진 클래스를 예측으로 삼는다. 
- RandomForestRegressor: 회귀 일때는 단수히 각 틔의 애측을 평균 합니다.

 
## 랜덤포레스트분류(RandomForestClassifier) 




In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
#입력 요소에 대한 특징 추출 
rf.fit(train_input, train_target)
print(rf.feature_importances_)

#과대 적합 
#훈련 데이터에 대해서는 높은 점수 , 테스트 데이터에 대해서는 낮은 점수 
#0.9974503966084433 0.8887848893166506

[0.23167441 0.50039841 0.26792718]


In [4]:
# n_jobs=-1 : CPU 코어 설정 
# oob_score = True : 부트 스트랩 샘플에 포함되지 않는 나머지 샘플로 훈련한 결정 트리를 평가 

#rf = RandomForestClassifier(oob_score=False, n_jobs=-1, random_state=42)

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)

# : 랜덤 포레스트는 각 결정 트리에 OOB 점수를 평균하여 출력 
print(rf.oob_score_) #0.8934000384837406 

0.8934000384837406


## 엑스트라트리
- 부트스트랩 샘플을 사용하지 않고 각 결정 트리를 만들때 전체 
- 노드를 분할 할때 무작위 분활한다.


In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

#교차검증 
#return_train_score = True : 검증 점수, 훈련 세트에 대한 점수도 리턴 
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

#[알코올,      당도,       산도       ] -> 당도가 높음 (레드와인 과 ) 
#[0.20183568 0.52242907 0.27573525]

#이전 decisiontree(5-1) 
#[0.15210271 0.70481604 0.14308125]

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

깊이 야은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법 
긱본적으로 깊이가 3인 결정 트리를 100개 사용
깊이가 얕은 결정트리를 사용하기 때문에 과대 적합에강하고 일반적으로 높은 일반화 산을을 가짐
결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동
학습률 매개변수로 속도를 조절 

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#고대적합이 되지 않음
# 0.8881086892152563 0.8720430147331015


0.8881086892152563 0.8720430147331015


In [16]:
# 결정트리의 갯수 : n_estimators=500 , 기본값 100 
# 학습률 : learning_rate=0.2 , 기본값 0.1 
# 
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과대접합이 억제 보임   
# 0.9464595437171814 0.8780082549788999 

0.9464595437171814 0.8780082549788999


In [17]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
#당도에대해 집중함. 

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅
정형데이터를 당루는 미신러닝 알고리즘 중에 가장인가 높은 알고리즘 
입력 특성을 256 개 구간으로 나눔으로서 노드를 분활시 최적의 분활을 빠르게할 수 있다.
256개 구간 중에서 하나를 누락된 값을 위해 배정 
입력 누락값에 대해 전처리가 필요없음


In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.experimental import enable_hist_gradient_boosting


hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#30.9321723946453317 0.8801241948619236
#과대적합을 억제하면서 그레이디언트 부싕보다 조금 높은 성능을 제공
#트리를 사용하면서 특성 중요도를 뽑을수 있는데 -> 실제로는 P272 특정중요도 는 넣지 안는다 (git 오류패이지에서 확인가능 )

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)

print(result.importances_mean)

#


[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [14]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [15]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

ModuleNotFoundError: No module named 'lightgbm'